For now on, I'll need a faster way to manage the database. This test will check accesibility for datasets with HDF5


In [2]:
!pip install simpleITK

     |████████████████████████████████| 47.4MB 86kB/s 


In [ ]:
!git config --global user.email 'obandomarcos96@gmail.com'
!git config --global user.password 'Quovadis96'

In [12]:
import os
os.chdir('/content/drive/MyDrive/Balseiro/Maestría/Implementación/DeepOPT')
# import some libraries
import os,time, sys
sys.path.append('Utilities/')

import numpy as np
import h5py as h5
%load_ext autoreload
%autoreload 1
import matplotlib.pyplot as plt
import pandas as pd
%aimport DataLoading
DL = DataLoading

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
f140114_5dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140114_5dpf"  # 5 days post-fertilization
f140117_3dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140117_3dpf"  # 3 days post-fertilization
f140115_1dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140315_1dpf"  # 1 days post-fertilization

f140315_3dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140315_3dpf"     # 3 days post-fertilization
f140415_5dpf_4X = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140415_5dpf_4X"  # 5 days post-fertilization
f140419_5dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140519_5dpf"     # 5 days post-fertilization

f140714_5dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140714_5dpf"
f140827_3dpf_4X = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140827_3dpf_4X"
f140827_5dpf_4X = '/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140827_5dpf_4X'

folder_paths = [f140114_5dpf]
results_folder = '/content/drive/MyDrive/Balseiro/Maestría/Implementación/1-DL/Resultados/'

In [3]:
dfs = [DL.ZebraDataset(dataset, 'Datasets', 'Bassi') for dataset in folder_paths]

for df in dfs:
  
  # Cargo el dataset
  df.loadImages(sample = 'head')
  # Cargo las registraciones correspondientes
  df.loadRegTransforms()
  # Aplico las transformaciones para este dataset 
  df.applyRegistration(sample = 'head')

100%|██████████| 180/180 [00:11<00:00, 15.40it/s]


In [26]:
df.saveRegisteredDataset()

/usr/local/lib/python3.7/dist-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '140114_5dpf'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->values] [items->None]

  encoding=encoding,


ValueError: ignored

In [29]:
!git add Utilities/DataLoading.py Tests/MoDL_training_test.ipynb Tests/HDF5_dataset_test.ipynb; git commit -m '#9 option 1, saving datasets with built-in Pandas method'

fatal: pathspec 'Tests/MoDL_training_test' did not match any files

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@8ea729da9e88.(none)')


In [25]:
with h5.File(df.datasetFolder+'/'+'OPTdatasets.hdf5', 'a') as datasets_file:

  print(datasets_file['Bassi']['140114_5dpf'].keys())


<KeysViewHDF5 ['index', 'values']>
